# Preference Alignment with Direct Preference Optimization (DPO)

This notebook will guide you through the process of fine-tuning a language model using Direct Preference Optimization (DPO). We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with DPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p>
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Select a dataset that relates to a real-world use case you’re interested in, or use the model you trained in
        <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>

In [ ]:
# Install the requirements in Google Colab
!pip install transformers trl peft accelerate datasets bitsandbytes wandb huggingface_hub
# Authenticate to Hugging Face
from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [ ]:
!pip install --upgrade transformers trl peft accelerate datasets bitsandbytes wandb

## Import libraries


In [18]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig


## Format dataset

In [23]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
dataset = load_dataset(path="Danielbrdz/Barcenas-Medicina-DPO", split="train")

In [20]:
from IPython.display import display, HTML

display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/Danielbrdz/Barcenas-Medicina-DPO/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [24]:
def process_dataset(samples) -> dict[str, str]:
    # Replace null values with "" in each column
    return {
        "prompt": [samples["question"][i] if samples["question"][i] is not None else "" for i in range(len(samples["question"]))],
        "chosen": [samples["chosen"][i] if samples["chosen"][i] is not None else "" for i in range(len(samples["chosen"]))],
        "rejected": [samples["rejected"][i] if samples["rejected"][i] is not None else "" for i in range(len(samples["rejected"]))],
    }

original_columns = dataset.column_names

dataset = dataset.map(process_dataset, batched=True, remove_columns=original_columns)

print(dataset)


Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 10000
})


In [25]:

dataset[1]


{'chosen': 'Los glóbulos blancos son responsables de combatir infecciones y enfermedades en el cuerpo.',
 'rejected': 'Los glóbulos blancos son responsables de transportar oxígeno a los tejidos del cuerpo.',
 'prompt': '¿Cuál es la función principal de los glóbulos blancos en el sistema inmunológico?'}

## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 change the model to the path or repo id of the model you trained in <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>


In [26]:
# TODO: 🦁 change the model to the path or repo id of the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb)

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO-Medicina_es"
finetune_tags = ["smol-course", "module_2"]

Wandb

In [27]:
import wandb
import os

wandb.login()

wandb_project = "SmolLM2-FT-DPO-Medicina_es"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

wandb: Currently logged in as: da-qc. Use `wandb login --relogin` to force relogin


In [28]:
os.environ["WANDB_PROJECT"]

'SmolLM2-FT-DPO-Medicina_es'

## Train model with DPO

In [38]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=300,
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output",
    # Number of steps for learning rate warmup
    warmup_steps=100,
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to="wandb",
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    use_mps_device=device == "mps",
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    beta=0.1, # Moved beta parameter inside DPOConfig
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    max_length=1536,
)

In [39]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,

)

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [40]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,0.693100
2,0.693100
3,0.695700
4,0.689700
5,0.692600
6,0.694100
7,0.685500
8,0.691800
9,0.690400
10,0.691600


TrainOutput(global_step=300, training_loss=0.2599006153084338, metrics={'train_runtime': 667.4439, 'train_samples_per_second': 7.192, 'train_steps_per_second': 0.449, 'total_flos': 0.0, 'train_loss': 0.2599006153084338, 'epoch': 0.48})

In [41]:
# Save the model
trainer.save_model(f"./{finetune_name}")


In [92]:
dpo_model_path = "./SmolLM2-FT-DPO-Medicina_es/"
dpo_model = AutoModelForCausalLM.from_pretrained(dpo_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(dpo_model_path)

chat_template = "{content}"

# Let's test the base model before training
prompt = "¿Cuál es el tratamiento recomendado para la gripe?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = prompt

# Tokenization with attention
inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True).to(device)

# Generate response
output = dpo_model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=512,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=0.7
)

response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


¿Cuál es el tratamiento recomendado para la gripe? El tratamiento que puede ayudar en el diagnóstico y la evaluación del tratamiento del tratamiento del gripe


In [43]:
# Save to the huggingface hub if login (HF_TOKEN is set)
trainer.push_to_hub(tags=finetune_tags)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.26k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/daqc/SmolLM2-FT-DPO-Medicina_es/commit/5ae79a8aa120d43796e11b4e8e3b8d85364ba172', commit_message='End of training', commit_description='', oid='5ae79a8aa120d43796e11b4e8e3b8d85364ba172', pr_url=None, repo_url=RepoUrl('https://huggingface.co/daqc/SmolLM2-FT-DPO-Medicina_es', endpoint='https://huggingface.co', repo_type='model', repo_id='daqc/SmolLM2-FT-DPO-Medicina_es'), pr_revision=None, pr_num=None)

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `DPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.

# 📥 Uploaded Model and Wandb Logs  

- **🤖 Hugging Face Repository**: [SmolLM2-FT-DPO-Medicina_es](https://huggingface.co/daqc/SmolLM2-FT-DPO-Medicina_es)  
- **📊 Weights and Biases Run**: [Training Logs and Metrics](https://wandb.ai/da-qc/SmolLM2-FT-DPO-Medicina_es/runs/7mmqyef0?nw=nwuserdaqc)